In [143]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [144]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [145]:
subject = 'Acre - Consumo de Cimento (t)'
start_index = 0
split_index = 202 #Referente aos 230 meses de input  
train_split = split_index + 1 - 12*3

In [146]:
data = pd.read_csv('2003_mo_model_input_AC.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Unnamed: 0,Acre - IDH Longevidade,Acre - IDH Renda,Acre - Desemprego,Acre - IDH,Acre - PIB - Estadual,Acre - PIB - Construção Civil,Acre - PIB - Per Capita,Acre - PIB - Preços de Mercado,Acre - IDH Educacao,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),Contratação SFH (Bi R$),Contratação Jurídico Comercial (Bi R$),Contratação Jurídico FGTS (Bi R$),Contratação Jurídico Livre (Bi R$),Contratação Jurídico SFH (Bi R$),Direcionamento Aquisição (R$),Direcionamento Construção (R$),Direcionamentos Reformas e Ampliações (R$),Direcionamento Aplicação (R$),Ativos Problemáticos - Comercial (R$),Ativos Problemáticos - Home equity (R$),Ativos Problemáticos - Livre (R$),Ativos Problemáticos - SFH (R$),Ativos Problemáticos - FGTS (R$),Ativos Problemáticos P.Jurídica - Comercial (R$),Ativos Problemáticos P.Jurídica - Home equity (R$),Ativos Problemáticos P.Jurídica - Livre (R$),Ativos Problemáticos P.Jurídica - SFH (R$),Carteira de Crédito - Comercial (R$),Carteira de Crédito – Home equity (R$),Carteira de Crédito – Livre (R$),Carteira de Crédito - FGTS (R$),Carteira de Crédito - SFH (R$),Carteira de Crédito P.Jurídica - Comercial (R$),Carteira de Crédito P.Jurídica - Home equity (R$),Carteira de Crédito P.Jurídica - Livre (R$),Carteira de Crédito P.Jurídica - FGTS (R$),Inadimplência - SFH (%),Inadimplência - FGTS (%),Inadimplência - Livre (%),Inadimplência - Comercial (%),Inadimplência - Home equity (%),Inadimplência P.Jurídica - SFH (%),Inadimplência P.Jurídica - FGTS (%),Inadimplência P.Jurídica - Livre (%),Inadimplência P.Jurídica - Comercial (%),Poupança e Empréstimo - SBPE (R$),Títulos - LH (R$),Títulos – LIG (R$),Títulos – CRI (R$),Títulos – LCI (R$),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Acre - Consumo de Cimento (t)
0,2003-1,0.773065,0.651726,8.722906,0.691320,7.285707e+06,413409.255032,9.318417,6.587220e+06,0.643805,5.020927e+07,2.014625e+08,1.542506e+09,9.248956e+08,3.117876e+09,1.486875e+08,5.214374e+07,4.235269e+08,5.871672e+08,1.300935e+11,1.039573e+10,7.041098e+08,1.145754e+11,8.008715e+06,4.355660e+08,5.284752e+08,3.647348e+09,4.601248e+09,1.680008e+07,3.043953e+08,3.653415e+08,7.417182e+08,4.206722e+08,9.143676e+09,7.702509e+10,6.664510e+10,7.035792e+10,4.872928e+08,2.571144e+09,1.970044e+10,2.021690e+10,1.557727,2.468118,1.603214,0.300795,3.988206,0.833114,0.356568,1.481000,0.612453,1.540380e+11,1.211245e+09,8.107262e+09,1.921789e+10,8.202881e+10,0.724032,11.520143,1.639718,1.036534,1.772069e+08,7.330309e+06,0.969649,3.260
1,2003-2,0.773454,0.651847,8.718028,0.691617,7.294881e+06,413607.843560,9.319990,6.590338e+06,0.644270,5.018273e+07,2.015196e+08,1.542506e+09,9.267621e+08,3.118866e+09,1.486875e+08,5.214374e+07,4.235269e+08,5.871672e+08,1.300935e+11,1.039573e+10,7.041098e+08,1.145754e+11,8.008715e+06,4.373259e+08,5.322642e+08,3.662680e+09,4.601248e+09,1.680008e+07,3.043953e+08,3.653415e+08,7.417182e+08,4.206722e+08,9.149335e+09,7.702509e+10,6.664510e+10,7.035792e+10,4.872928e+08,2.571144e+09,1.970044e+10,2.021690e+10,1.557727,2.467045,1.603118,0.300795,3.991229,0.833114,0.356568,1.481000,0.612453,1.540380e+11,1.211245e+09,8.107262e+09,1.921789e+10,8.222716e+10,0.690297,11.189862,1.378899,0.993449,1.773884e+08,7.335910e+06,0.950783,2.191
2,2003-3,0.773843,0.651968,8.713149,0.691914,7.304055e+06,413806.432089,9.321563,6.593455e+06,0.644734,5.016725e+07,2.015529e+08,1.542506e+09,9.290952e+08,3.119443e+09,1.486875e+08,5.214374e+07,4.235269e+08,5.871672e+08,1.300935e+11,1.039573e+10,7.041098e+08,1.145754e+11,8.008715e+06,4.383525e+08,5.344744e+08,3.671623e+09,4.601248e+09,1.680008e+07,3.043953e+08,3.653415e+08,7.417182e+08,4.206722e+08,9.152636e+09,7.702509e+10,6.664510e+10,7.035792e+10,4.872928e+08,2.571144e+09,1.970044e+10,2.021690e+10,1.557727,2.466420,1.603063,0.300795,3.992993,0.833114,0.356568,1.481000,0.612453,1.540380e+11,1.211245e+09,8.

In [147]:
input_data = data.iloc[start_index:split_index + 1,1:-1]
# # input_data = data.iloc[:split_index + 1,:]
mean = np.mean(input_data, axis=0)
stddev = np.std(input_data, axis=0)
input_data = (input_data - mean) / stddev
input_data

,Acre - IDH Longevidade,Acre - IDH Renda,Acre - Desemprego,Acre - IDH,Acre - PIB - Estadual,Acre - PIB - Construção Civil,Acre - PIB - Per Capita,Acre - PIB - Preços de Mercado,Acre - IDH Educacao,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),Contratação SFH (Bi R$),Contratação Jurídico Comercial (Bi R$),Contratação Jurídico FGTS (Bi R$),Contratação Jurídico Livre (Bi R$),Contratação Jurídico SFH (Bi R$),Direcionamento Aquisição (R$),Direcionamento Construção (R$),Direcionamentos Reformas e Ampliações (R$),Direcionamento Aplicação (R$),Ativos Problemáticos - Comercial (R$),Ativos Problemáticos - Home equity (R$),Ativos Problemáticos - Livre (R$),Ativos Problemáticos - SFH (R$),Ativos Problemáticos - FGTS (R$),Ativos Problemáticos P.Jurídica - Comercial (R$),Ativos Problemáticos P.Jurídica - Home equity (R$),Ativos Problemáticos P.Jurídica - Livre (R$),Ativos Problemáticos P.Jurídica - SFH (R$),Carteira de Crédito - Comercial (R$),Carteira de Crédito – Home equity (R$),Carteira de Crédito – Livre (R$),Carteira de Crédito - FGTS (R$),Carteira de Crédito - SFH (R$),Carteira de Crédito P.Jurídica - Comercial (R$),Carteira de Crédito P.Jurídica - Home equity (R$),Carteira de Crédito P.Jurídica - Livre (R$),Carteira de Crédito P.Jurídica - FGTS (R$),Inadimplência - SFH (%),Inadimplência - FGTS (%),Inadimplência - Livre (%),Inadimplência - Comercial (%),Inadimplência - Home equity (%),Inadimplência P.Jurídica - SFH (%),Inadimplência P.Jurídica - FGTS (%),Inadimplência P.Jurídica - Livre (%),Inadimplência P.Jurídica - Comercial (%),Poupança e Empréstimo - SBPE (R$),Títulos - LH (R$),Títulos – LIG (R$),Títulos – CRI (R$),Títulos – LCI (R$),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%)
0,-2.409647,-2.350344,-0.866314,-2.228425,-1.822062,0.546215,-0.273640,-2.397582,-2.029666,1.869243,-1.363288,-0.328676,-1.292782,-1.133271,-0.455204,-0.309454,-0.179645,-0.157346,-1.029640,-0.904029,0.048164,-1.409358,-0.362145,-0.592656,-0.831086,-1.461839,-1.035363,-0.559280,-0.356321,-0.328621,-0.273458,-0.406297,-1.306377,1.693230,-0.882165,-1.201500,-1.749536,-0.336551,-0.313771,-0.509840,1.335105,0.733688,1.099345,-1.247441,-0.050789,-0.221470,-0.311447,-0.306607,-0.259125,-1.614710,-0.108765,1.219134,-1.060724,-0.818749,2.776604,4.498332,2.068834,2.666773,-2.062964,-2.463735,3.185074
1,-2.366241,-2.312597,-0.869262,-2.191719,-1.802070,0.562681,-0.250398,-2.354477,-2.001560,1.863972,-1.361516,-0.328676,-1.291119,-1.132415,-0.455204,-0.309454,-0.179645,-0.157346,-1.029640,-0.904029,0.048164,-1.409358,-0.362145,-0.588299,-0.826280,-1.455767,-1.035363,-0.559280,-0.356321,-0.328621,-0.273458,-0.406297,-1.300714,1.693230,-0.882165,-1.201500,-1.749536,-0.336551,-0.313771,-0.509840,1.335105,0.729147,1.098971,-1.247441,-0.047503,-0.221470,-0.311447,-0.306607,-0.259125,-1.614710,-0.108765,1.219134,-1.060724,-0.813998,2.402991,4.320105,1.283578,2.420378,-2.036160,-2.425826,3.029813
2,-2.322835,-2.274850,-0.872210,-2.155013,-1.782078,0.579147,-0.227156,-2.311373,-1.973454,1.860898,-1.360482,-0.328676,-1.289041,-1.131916,-0.455204,-0.309454,-0.179645,-0.157346,-1.029640,-0.904029,0.048164,-1.409358,-0.362145,-0.585758,-0.823476,-1.452225,-1.035363,-0.559280,-0.356321,-0.328621,-0.273458,-0.406297,-1.297410,1.693230,-0.882165,-1.201500,-1.749536,-0.336551,-0.313771,-0.509840,1.335105,0.726498,1.098752,-1.247441,-0.045586,-0.221470,-0.311447,-0.306607,-0.259125,-1.614710,-0.108765,1.219134,-1.060724,-0.811226,2.174668,4.120946,2.925685,2.303546,-2.009356,-2.387918,2.927348
3,-2.279429,-2.237104,-0.875158,-2.118307,-1.762086,0.595613,-0.203914,-2.268269,-1.945347,1.860569,-1.360372,-0.328676,-1.287876,-1.131862,-0.455204,-0.309454,-0.179645,-0.157346,-1.029640,-0.904029,0.048164,-1.409358,-0.362145,-0.585485,-0.823176,-1.451846,-1.035363,-0.559280,-0.356321,-0.328621,-0.273458,-0.406297,

In [148]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0       2.191
1       2.154
2       2.643
3       2.546
4       3.040
        ...  
236    18.573
237    15.493
238    15.493
239       NaN
240       NaN
Name: Acre - Consumo de Cimento (t), Length: 241, dtype: float64

In [149]:
# input para treinamento
train_input = input_data.iloc[:train_split - start_index]
train_input

,Acre - IDH Longevidade,Acre - IDH Renda,Acre - Desemprego,Acre - IDH,Acre - PIB - Estadual,Acre - PIB - Construção Civil,Acre - PIB - Per Capita,Acre - PIB - Preços de Mercado,Acre - IDH Educacao,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),Contratação SFH (Bi R$),Contratação Jurídico Comercial (Bi R$),Contratação Jurídico FGTS (Bi R$),Contratação Jurídico Livre (Bi R$),Contratação Jurídico SFH (Bi R$),Direcionamento Aquisição (R$),Direcionamento Construção (R$),Direcionamentos Reformas e Ampliações (R$),Direcionamento Aplicação (R$),Ativos Problemáticos - Comercial (R$),Ativos Problemáticos - Home equity (R$),Ativos Problemáticos - Livre (R$),Ativos Problemáticos - SFH (R$),Ativos Problemáticos - FGTS (R$),Ativos Problemáticos P.Jurídica - Comercial (R$),Ativos Problemáticos P.Jurídica - Home equity (R$),Ativos Problemáticos P.Jurídica - Livre (R$),Ativos Problemáticos P.Jurídica - SFH (R$),Carteira de Crédito - Comercial (R$),Carteira de Crédito – Home equity (R$),Carteira de Crédito – Livre (R$),Carteira de Crédito - FGTS (R$),Carteira de Crédito - SFH (R$),Carteira de Crédito P.Jurídica - Comercial (R$),Carteira de Crédito P.Jurídica - Home equity (R$),Carteira de Crédito P.Jurídica - Livre (R$),Carteira de Crédito P.Jurídica - FGTS (R$),Inadimplência - SFH (%),Inadimplência - FGTS (%),Inadimplência - Livre (%),Inadimplência - Comercial (%),Inadimplência - Home equity (%),Inadimplência P.Jurídica - SFH (%),Inadimplência P.Jurídica - FGTS (%),Inadimplência P.Jurídica - Livre (%),Inadimplência P.Jurídica - Comercial (%),Poupança e Empréstimo - SBPE (R$),Títulos - LH (R$),Títulos – LIG (R$),Títulos – CRI (R$),Títulos – LCI (R$),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%)
0,-2.409647,-2.350344,-0.866314,-2.228425,-1.822062,0.546215,-0.273640,-2.397582,-2.029666,1.869243,-1.363288,-0.328676,-1.292782,-1.133271,-0.455204,-0.309454,-0.179645,-0.157346,-1.029640,-0.904029,0.048164,-1.409358,-0.362145,-0.592656,-0.831086,-1.461839,-1.035363,-0.559280,-0.356321,-0.328621,-0.273458,-0.406297,-1.306377,1.693230,-0.882165,-1.201500,-1.749536,-0.336551,-0.313771,-0.509840,1.335105,0.733688,1.099345,-1.247441,-0.050789,-0.221470,-0.311447,-0.306607,-0.259125,-1.614710,-0.108765,1.219134,-1.060724,-0.818749,2.776604,4.498332,2.068834,2.666773,-2.062964,-2.463735,3.185074
1,-2.366241,-2.312597,-0.869262,-2.191719,-1.802070,0.562681,-0.250398,-2.354477,-2.001560,1.863972,-1.361516,-0.328676,-1.291119,-1.132415,-0.455204,-0.309454,-0.179645,-0.157346,-1.029640,-0.904029,0.048164,-1.409358,-0.362145,-0.588299,-0.826280,-1.455767,-1.035363,-0.559280,-0.356321,-0.328621,-0.273458,-0.406297,-1.300714,1.693230,-0.882165,-1.201500,-1.749536,-0.336551,-0.313771,-0.509840,1.335105,0.729147,1.098971,-1.247441,-0.047503,-0.221470,-0.311447,-0.306607,-0.259125,-1.614710,-0.108765,1.219134,-1.060724,-0.813998,2.402991,4.320105,1.283578,2.420378,-2.036160,-2.425826,3.029813
2,-2.322835,-2.274850,-0.872210,-2.155013,-1.782078,0.579147,-0.227156,-2.311373,-1.973454,1.860898,-1.360482,-0.328676,-1.289041,-1.131916,-0.455204,-0.309454,-0.179645,-0.157346,-1.029640,-0.904029,0.048164,-1.409358,-0.362145,-0.585758,-0.823476,-1.452225,-1.035363,-0.559280,-0.356321,-0.328621,-0.273458,-0.406297,-1.297410,1.693230,-0.882165,-1.201500,-1.749536,-0.336551,-0.313771,-0.509840,1.335105,0.726498,1.098752,-1.247441,-0.045586,-0.221470,-0.311447,-0.306607,-0.259125,-1.614710,-0.108765,1.219134,-1.060724,-0.811226,2.174668,4.120946,2.925685,2.303546,-2.009356,-2.387918,2.927348
3,-2.279429,-2.237104,-0.875158,-2.118307,-1.762086,0.595613,-0.203914,-2.268269,-1.945347,1.860569,-1.360372,-0.328676,-1.287876,-1.131862,-0.455204,-0.309454,-0.179645,-0.157346,-1.029640,-0.904029,0.048164,-1.409358,-0.362145,-0.585485,-0.823176,-1.451846,-1.035363,-0.559280,-0.356321,-0.328621,-0.273458,-0.406297,

In [150]:
# Alvo para treinamento
train_target = target_data.iloc[start_index:train_split]
train_target

0       2.191000
1       2.154000
2       2.643000
3       2.546000
4       3.040000
         ...    
162    12.974077
163    12.460462
164    11.946846
165    11.433231
166    10.919615
Name: Acre - Consumo de Cimento (t), Length: 167, dtype: float64

In [151]:
#input de test (Ano 2021)
test_input = input_data.iloc[train_split - start_index:]
test_input

,Acre - IDH Longevidade,Acre - IDH Renda,Acre - Desemprego,Acre - IDH,Acre - PIB - Estadual,Acre - PIB - Construção Civil,Acre - PIB - Per Capita,Acre - PIB - Preços de Mercado,Acre - IDH Educacao,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),Contratação SFH (Bi R$),Contratação Jurídico Comercial (Bi R$),Contratação Jurídico FGTS (Bi R$),Contratação Jurídico Livre (Bi R$),Contratação Jurídico SFH (Bi R$),Direcionamento Aquisição (R$),Direcionamento Construção (R$),Direcionamentos Reformas e Ampliações (R$),Direcionamento Aplicação (R$),Ativos Problemáticos - Comercial (R$),Ativos Problemáticos - Home equity (R$),Ativos Problemáticos - Livre (R$),Ativos Problemáticos - SFH (R$),Ativos Problemáticos - FGTS (R$),Ativos Problemáticos P.Jurídica - Comercial (R$),Ativos Problemáticos P.Jurídica - Home equity (R$),Ativos Problemáticos P.Jurídica - Livre (R$),Ativos Problemáticos P.Jurídica - SFH (R$),Carteira de Crédito - Comercial (R$),Carteira de Crédito – Home equity (R$),Carteira de Crédito – Livre (R$),Carteira de Crédito - FGTS (R$),Carteira de Crédito - SFH (R$),Carteira de Crédito P.Jurídica - Comercial (R$),Carteira de Crédito P.Jurídica - Home equity (R$),Carteira de Crédito P.Jurídica - Livre (R$),Carteira de Crédito P.Jurídica - FGTS (R$),Inadimplência - SFH (%),Inadimplência - FGTS (%),Inadimplência - Livre (%),Inadimplência - Comercial (%),Inadimplência - Home equity (%),Inadimplência P.Jurídica - SFH (%),Inadimplência P.Jurídica - FGTS (%),Inadimplência P.Jurídica - Livre (%),Inadimplência P.Jurídica - Comercial (%),Poupança e Empréstimo - SBPE (R$),Títulos - LH (R$),Títulos – LIG (R$),Títulos – CRI (R$),Títulos – LCI (R$),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%)
167,1.346318,0.936802,1.273215,1.284120,1.109854,-1.473368,-1.484877,0.660687,1.226873,-0.985677,-1.266879,-1.450047,1.412006,-1.363579,-1.332684,1.316091,-1.491837,-1.554919,1.106248,1.023900,-0.721665,1.238976,1.679964,1.542774,1.497170,1.364611,0.980153,1.050660,1.540667,1.501612,1.174089,1.375577,1.226234,-0.648643,1.217890,1.187709,1.175579,1.786750,-0.631364,-0.142949,0.714891,-1.726152,1.134880,1.723033,1.688803,0.680946,0.888762,1.063326,1.215858,0.883373,-0.990904,-0.665494,1.371826,1.474316,-0.976994,-0.587973,0.295211,-1.102753,1.034288,0.717292,-1.439110
168,1.339877,0.911132,1.262526,1.281337,1.111887,-1.473348,-1.499313,0.658768,1.224171,-1.026667,-1.330286,-1.471117,1.321927,-1.361320,-1.340444,1.342242,-1.499404,-1.543867,1.136112,1.058954,-0.776628,1.257006,1.679455,1.523142,1.486233,1.352172,1.000488,1.107162,1.597873,1.584622,1.258917,1.386905,1.209006,-0.699350,1.249007,1.193748,1.228522,1.799085,-0.704999,-0.232239,0.658407,-1.713312,1.110942,1.704586,1.678947,0.745240,0.872658,1.131442,1.240401,0.900166,-1.034161,-0.665494,1.381520,1.460667,-1.029191,-0.588298,-0.516250,-1.219794,1.045301,0.705820,-1.440889
169,1.328626,0.916259,1.257646,1.288588,1.109593,-1.458972,-1.506238,0.654846,1.236353,-1.101280,-1.328942,-1.487877,1.340123,-1.336289,-1.347194,1.435454,-1.534533,-1.540367,1.166963,1.094564,-0.832746,1.274938,1.672059,1.500675,1.473099,1.338268,1.021881,1.176504,1.653322,1.656483,1.344009,1.399902,1.192505,-0.751575,1.280294,1.199893,1.282782,1.811406,-0.783338,-0.321558,0.589375,-1.702286,1.079916,1.673980,1.658110,0.810398,0.894137,1.209424,1.254558,0.917989,-1.073663,-0.665494,1.388190,1.446106,-1.074478,-0.584102,-0.383931,-1.155351,1.056232,0.690782,-1.520966
170,1.317375,0.921387,1.252766,1.295839,1.107300,-1.444596,-1.513164,0.650923,1.248536,-1.166990,-1.299737,-1.503728,1.339132,-1.313700,-1.345500,1.410110,-1.542351,-1.515319,1.197572,1.130373,-0.890030,1.291405,1.666417,1.474594,1.458071,1.321789,1.043829,1.253320,1.704440,1.725710,1.429016,1.418128,1.176311,-0.806226,1.311480,1.205958,1.337822,1.821620,-0.861829,-0.412087,0.474650,-1.699106

In [152]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[train_split:split_index + 1]
test_target

167    10.406
168     9.081
169     9.545
170    10.130
171    14.451
172    13.218
173    13.552
174    15.826
175    14.869
176    14.734
177    12.754
178    11.925
179    13.114
180     8.920
181     9.724
182     9.737
183    10.136
184    14.403
185    12.871
186    15.749
187    14.269
188    15.367
189    14.595
190     9.811
191    11.188
192    10.381
193     8.080
194    11.154
195    12.508
196    12.126
197    14.496
198    16.723
199    15.253
200    16.531
201    15.206
202    10.160
Name: Acre - Consumo de Cimento (t), dtype: float64

In [153]:
def validation_splitter(df, div_factor, add_factor):
    split_factor = len(df)//div_factor
    positions_to_drop_index = []
    positions_to_drop = []
    for i in range(split_factor):
        pos = df.shape[0] - (i*6 + 1)
        positions_to_drop_index.append(pos + add_factor)
        positions_to_drop.append(pos)
    
    return df.drop(positions_to_drop_index), df.iloc[positions_to_drop]

In [157]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0, start_index=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(128, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    train, train_val = validation_splitter(train_input, 6, start_index)
    target,target_val = validation_splitter(train_target, 6, start_index)
#     display(train)
#     display(train_val)
#     display(target)
#     display(target_val)
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train, 
                        target, 
                        epochs=10000,
                        validation_data=(train_val, target_val),
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [158]:
def get_a_good_seed(train_input, train_target, test_target, test_input, start_index=0):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(25)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, want_verbose=0, seed=seed, start_index=start_index)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [159]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input, start_index)
print(f"\n\nfinal_seed: {winner_seed}")

[3448415865, 1313631002, 3000725254, 3125039588, 3381414146, 3271373770, 392459201, 761475653, 3249571578, 2576356869, 2509147785, 1948242181, 1288041963, 3148354890, 3356310162, 2776657395, 605578477, 800363479, 2628050114, 349937761, 1602803969, 1727633450, 1501684248, 1835812938, 2048562526]


Step: 0 ___________________________________________
val_loss: 1.394444227218628
winner_seed: 3448415865


Step: 1 ___________________________________________
val_loss: 1.595787763595581


Step: 2 ___________________________________________
val_loss: 1.667346477508545


Step: 3 ___________________________________________
val_loss: 1.934644341468811


Step: 4 ___________________________________________
val_loss: 1.700308084487915


Step: 5 ___________________________________________
val_loss: 1.6704808473587036


Step: 6 ___________________________________________


2023-10-12 13:19:09.639064: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled


val_loss: 1.4159274101257324


Step: 7 ___________________________________________
val_loss: 1.3718992471694946
winner_seed: 761475653


Step: 8 ___________________________________________
val_loss: 1.6740838289260864


Step: 9 ___________________________________________
val_loss: 1.492985486984253


Step: 10 ___________________________________________
val_loss: 1.5875450372695923


Step: 11 ___________________________________________
val_loss: 1.4990743398666382


Step: 12 ___________________________________________
val_loss: 1.7889525890350342


Step: 13 ___________________________________________
val_loss: 1.6956489086151123


Step: 14 ___________________________________________
val_loss: 1.6628344058990479


Step: 15 ___________________________________________
val_loss: 1.6833425760269165


Step: 16 ___________________________________________
val_loss: 1.355094075202942
winner_seed: 605578477


Step: 17 ___________________________________________
val_loss: 1.43351149559021


Step: 

2023-10-12 13:26:23.670866: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled
2023-10-12 13:26:23.706579: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled


val_loss: 1.7397878170013428


final_seed: 605578477


In [160]:
trained_model, history = neural_network_model(train_input, 
                                              train_target, 
                                              want_verbose=1, 
                                              seed=winner_seed,
                                              start_index=start_index)

Epoch 1/10000
5/5 [==============================] - 0s 26ms/step - loss: 120.2273 - val_loss: 67.9269
Epoch 2/10000
5/5 [==============================] - 0s 5ms/step - loss: 38.9765 - val_loss: 58.7190
Epoch 3/10000
5/5 [==============================] - 0s 4ms/step - loss: 17.7594 - val_loss: 4.4076
Epoch 4/10000
5/5 [==============================] - 0s 5ms/step - loss: 7.9718 - val_loss: 8.1684
Epoch 5/10000
5/5 [==============================] - 0s 4ms/step - loss: 7.7014 - val_loss: 10.0386
Epoch 6/10000
5/5 [==============================] - 0s 4ms/step - loss: 10.6624 - val_loss: 5.4174
Epoch 7/10000
5/5 [==============================] - 0s 4ms/step - loss: 7.3842 - val_loss: 2.7014
Epoch 8/10000
5/5 [==============================] - 0s 4ms/step - loss: 6.9307 - val_loss: 7.8914
Epoch 9/10000
5/5 [==============================] - 0s 4ms/step - loss: 7.0679 - val_loss: 4.2966
Epoch 10/10000
5/5 [==============================] - 0s 5ms/step - loss: 7.2263 - val_loss: 2.9614


Epoch 83/10000
5/5 [==============================] - 0s 4ms/step - loss: 6.7826 - val_loss: 1.5631
Epoch 84/10000
5/5 [==============================] - 0s 4ms/step - loss: 6.4244 - val_loss: 3.4965
Epoch 85/10000
5/5 [==============================] - 0s 4ms/step - loss: 5.8955 - val_loss: 2.9438
Epoch 86/10000
5/5 [==============================] - 0s 5ms/step - loss: 5.9820 - val_loss: 13.5071
Epoch 87/10000
5/5 [==============================] - 0s 4ms/step - loss: 8.3703 - val_loss: 2.7202
Epoch 88/10000
5/5 [==============================] - 0s 5ms/step - loss: 5.8924 - val_loss: 2.5574
Epoch 89/10000
5/5 [==============================] - 0s 4ms/step - loss: 6.1836 - val_loss: 2.7054
Epoch 90/10000
5/5 [==============================] - 0s 5ms/step - loss: 5.9443 - val_loss: 2.7904
Epoch 91/10000
5/5 [==============================] - 0s 5ms/step - loss: 6.2932 - val_loss: 3.1225
Epoch 92/10000
5/5 [==============================] - 0s 4ms/step - loss: 6.4660 - val_loss: 6.5449

5/5 [==============================] - 0s 4ms/step - loss: 6.0414 - val_loss: 2.1383
Epoch 165/10000
5/5 [==============================] - 0s 4ms/step - loss: 5.9112 - val_loss: 5.3359
Epoch 166/10000
5/5 [==============================] - 0s 4ms/step - loss: 6.4600 - val_loss: 3.2435
Epoch 167/10000
5/5 [==============================] - 0s 4ms/step - loss: 5.8043 - val_loss: 3.9927
Epoch 168/10000
5/5 [==============================] - 0s 5ms/step - loss: 5.7557 - val_loss: 3.1674
Epoch 169/10000
5/5 [==============================] - 0s 4ms/step - loss: 5.7294 - val_loss: 4.4804
Epoch 170/10000
5/5 [==============================] - 0s 4ms/step - loss: 5.9419 - val_loss: 4.3181
Epoch 171/10000
5/5 [==============================] - 0s 4ms/step - loss: 5.8786 - val_loss: 4.1288
Epoch 172/10000
5/5 [==============================] - 0s 4ms/step - loss: 5.9855 - val_loss: 4.1940
Epoch 173/10000
5/5 [==============================] - 0s 4ms/step - loss: 6.3678 - val_loss: 2.7764
Epoch 

5/5 [==============================] - 0s 4ms/step - loss: 5.4246 - val_loss: 2.3103
Epoch 246/10000
5/5 [==============================] - 0s 5ms/step - loss: 5.8311 - val_loss: 2.5195
Epoch 247/10000
5/5 [==============================] - 0s 4ms/step - loss: 5.6771 - val_loss: 2.6734
Epoch 248/10000
5/5 [==============================] - 0s 4ms/step - loss: 5.5326 - val_loss: 2.5060
Epoch 249/10000
5/5 [==============================] - 0s 4ms/step - loss: 5.7780 - val_loss: 2.5025
Epoch 250/10000
5/5 [==============================] - 0s 5ms/step - loss: 5.5406 - val_loss: 3.5887
Epoch 251/10000
5/5 [==============================] - 0s 4ms/step - loss: 5.5593 - val_loss: 3.3527
Epoch 252/10000
5/5 [==============================] - 0s 4ms/step - loss: 5.9176 - val_loss: 2.2544
Epoch 253/10000
5/5 [==============================] - 0s 4ms/step - loss: 6.1658 - val_loss: 2.0031
Epoch 254/10000
5/5 [==============================] - 0s 5ms/step - loss: 5.9559 - val_loss: 1.9544
Epoch 

5/5 [==============================] - 0s 4ms/step - loss: 5.3295 - val_loss: 3.2532
Epoch 327/10000
5/5 [==============================] - 0s 4ms/step - loss: 5.1209 - val_loss: 3.3486
Epoch 328/10000
5/5 [==============================] - 0s 4ms/step - loss: 5.3385 - val_loss: 2.5975
Epoch 329/10000
5/5 [==============================] - 0s 4ms/step - loss: 5.1389 - val_loss: 2.5512
Epoch 330/10000
5/5 [==============================] - 0s 4ms/step - loss: 5.2071 - val_loss: 3.1764
Epoch 331/10000
5/5 [==============================] - 0s 4ms/step - loss: 5.0526 - val_loss: 4.0413
Epoch 332/10000
5/5 [==============================] - 0s 4ms/step - loss: 5.3662 - val_loss: 2.5821
Epoch 333/10000
5/5 [==============================] - 0s 4ms/step - loss: 5.7968 - val_loss: 6.4992
Epoch 334/10000
5/5 [==============================] - 0s 4ms/step - loss: 5.3047 - val_loss: 2.3245
Epoch 335/10000
5/5 [==============================] - 0s 4ms/step - loss: 5.4468 - val_loss: 4.3787
Epoch 

...

5/5 [==============================] - 0s 4ms/step - loss: 3.7113 - val_loss: 4.4898
Epoch 1135/10000
5/5 [==============================] - 0s 4ms/step - loss: 3.9403 - val_loss: 3.8423
Epoch 1136/10000
5/5 [==============================] - 0s 4ms/step - loss: 3.8443 - val_loss: 4.3834
Epoch 1137/10000
5/5 [==============================] - 0s 4ms/step - loss: 3.7228 - val_loss: 3.6944
Epoch 1138/10000
5/5 [==============================] - 0s 4ms/step - loss: 3.7751 - val_loss: 5.6090
Epoch 1139/10000
5/5 [==============================] - 0s 4ms/step - loss: 4.0201 - val_loss: 3.3619
Epoch 1140/10000
5/5 [==============================] - 0s 4ms/step - loss: 3.9069 - val_loss: 4.0486
Epoch 1141/10000
5/5 [==============================] - 0s 4ms/step - loss: 3.7437 - val_loss: 1.8331
Epoch 1142/10000
5/5 [==============================] - 0s 4ms/step - loss: 3.8161 - val_loss: 2.8313
Epoch 1143/10000
5/5 [==============================] - 0s 4ms/step - loss: 3.6569 - val_loss: 4.99

5/5 [==============================] - 0s 4ms/step - loss: 4.5509 - val_loss: 5.9102
Epoch 1215/10000
5/5 [==============================] - 0s 4ms/step - loss: 4.4841 - val_loss: 4.5543
Epoch 1216/10000
5/5 [==============================] - 0s 4ms/step - loss: 3.8570 - val_loss: 5.4135
Epoch 1217/10000
5/5 [==============================] - 0s 4ms/step - loss: 4.0747 - val_loss: 2.7375
Epoch 1218/10000
5/5 [==============================] - 0s 4ms/step - loss: 4.2290 - val_loss: 4.9765
Epoch 1219/10000
5/5 [==============================] - 0s 4ms/step - loss: 3.7714 - val_loss: 2.5106
Epoch 1220/10000
5/5 [==============================] - 0s 4ms/step - loss: 3.7424 - val_loss: 2.5314
Epoch 1221/10000
5/5 [==============================] - 0s 4ms/step - loss: 3.6668 - val_loss: 3.7971
Epoch 1222/10000
5/5 [==============================] - 0s 4ms/step - loss: 3.4312 - val_loss: 2.5734
Epoch 1223/10000
5/5 [==============================] - 0s 4ms/step - loss: 3.3324 - val_loss: 6.02

5/5 [==============================] - 0s 4ms/step - loss: 3.5594 - val_loss: 7.3812
Epoch 1295/10000
5/5 [==============================] - 0s 4ms/step - loss: 3.4314 - val_loss: 4.5483
Epoch 1296/10000
5/5 [==============================] - 0s 4ms/step - loss: 3.6265 - val_loss: 5.3506
Epoch 1297/10000
5/5 [==============================] - 0s 4ms/step - loss: 3.6388 - val_loss: 4.3264
Epoch 1298/10000
5/5 [==============================] - 0s 4ms/step - loss: 3.1672 - val_loss: 3.7274
Epoch 1299/10000
5/5 [==============================] - 0s 4ms/step - loss: 3.1244 - val_loss: 5.3463
Epoch 1300/10000
5/5 [==============================] - 0s 4ms/step - loss: 3.5974 - val_loss: 3.6460
Epoch 1301/10000
5/5 [==============================] - 0s 4ms/step - loss: 3.1978 - val_loss: 5.0286
Epoch 1302/10000
5/5 [==============================] - 0s 4ms/step - loss: 3.7731 - val_loss: 3.6357
Epoch 1303/10000
5/5 [==============================] - 0s 4ms/step - loss: 3.0289 - val_loss: 4.44

5/5 [==============================] - 0s 4ms/step - loss: 2.9666 - val_loss: 2.6717
Epoch 1375/10000
5/5 [==============================] - 0s 4ms/step - loss: 3.1938 - val_loss: 5.0912
Epoch 1376/10000
5/5 [==============================] - 0s 4ms/step - loss: 3.2988 - val_loss: 2.4302
Epoch 1377/10000
5/5 [==============================] - 0s 4ms/step - loss: 2.8871 - val_loss: 2.9449
Epoch 1378/10000
5/5 [==============================] - 0s 4ms/step - loss: 2.8981 - val_loss: 3.5614
Epoch 1379/10000
5/5 [==============================] - 0s 4ms/step - loss: 3.7364 - val_loss: 2.4938
Epoch 1380/10000
5/5 [==============================] - 0s 4ms/step - loss: 3.3657 - val_loss: 4.2317
Epoch 1381/10000
5/5 [==============================] - 0s 4ms/step - loss: 3.5709 - val_loss: 3.2463
Epoch 1382/10000
5/5 [==============================] - 0s 4ms/step - loss: 3.0229 - val_loss: 3.5041
Epoch 1383/10000
5/5 [==============================] - 0s 4ms/step - loss: 3.5130 - val_loss: 3.94

5/5 [==============================] - 0s 4ms/step - loss: 3.5502 - val_loss: 3.6359
Epoch 1455/10000
5/5 [==============================] - 0s 4ms/step - loss: 2.9384 - val_loss: 3.8735
Epoch 1456/10000
5/5 [==============================] - 0s 4ms/step - loss: 2.7925 - val_loss: 3.1227
Epoch 1457/10000
5/5 [==============================] - 0s 4ms/step - loss: 2.5582 - val_loss: 3.1382
Epoch 1458/10000
5/5 [==============================] - 0s 4ms/step - loss: 3.1425 - val_loss: 4.4089
Epoch 1459/10000
5/5 [==============================] - 0s 4ms/step - loss: 3.0471 - val_loss: 4.4539
Epoch 1460/10000
5/5 [==============================] - 0s 4ms/step - loss: 3.5498 - val_loss: 2.6479
Epoch 1461/10000
5/5 [==============================] - 0s 4ms/step - loss: 2.9491 - val_loss: 4.0256
Epoch 1462/10000
5/5 [==============================] - 0s 4ms/step - loss: 3.1734 - val_loss: 2.9453
Epoch 1463/10000
5/5 [==============================] - 0s 4ms/step - loss: 3.3067 - val_loss: 4.10

In [161]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    results_data = []

    for i in range(len(test_target)):
        prediction = model.predict(test_input.iloc[i:i+1])
        target = test_target[start_target + i]
        error = np.abs(prediction - target)
        errors.append(error)
        error_percent.append(error/target)
        results_data.append([f"Month-{i + 1}", 
                             prediction[0][0], 
                             target, 
                             error[0][0]])

    df_results = pd.DataFrame(results_data, columns=["Month", "Prediction", "Target", "Error"])

    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return df_results, mae, mape

In [162]:
df_results, mae, mape = mae_mape_calculator(trained_model, 
                                        test_input, 
                                        test_target, 167)
pd.set_option('display.max_columns', None)
df_results.T

1/1 [==============================] - 0s 12ms/step


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35
Month,Month-1,Month-2,Month-3,Month-4,Month-5,Month-6,Month-7,Month-8,Month-9,Month-10,Month-11,Month-12,Month-13,Month-14,Month-15,Month-16,Month-17,Month-18,Month-19,Month-20,Month-21,Month-22,Month-23,Month-24,Month-25,Month-26,Month-27,Month-28,Month-29,Month-30,Month-31,Month-32,Month-33,Month-34,Month-35,Month-36
Prediction,12.509471,11.173357,9.191566,6.510533,2.495927,-4.206169,-3.550532,-6.417638,-7.004304,-6.746214,-6.885063,-6.999981,-7.174329,-7.34915,-7.191216,-7.244228,-7.270323,-7.210356,-7.192362,-7.161911,-7.15646,-7.161361,-7.17206,-7.175457,-7.22932,-7.236347,-7.338739,-7.302138,-7.270134,-7.369872,-7.408483,-7.5022,-7.610399,-7.809597,-8.151068,-8.422524
Target,10.406,9.081,9.545,10.13,14.451,13.218,13.552,15.826,14.869,14.734,12.754,11.925,13.114,8.92,9.724,9.737,10.136,14.403,12.871,15.749,14.269,15.367,14.595,9.811,11.188,10.381,8.08,11.154,12.508,12.126,14.496,16.723,15.253,16.531,15.206,10.16
Error,2.103471,2.092357,0.353435,3.619467,11.955073,17.42417,17.102531,22.243637,21.873304,21.480213,19.639063,18.924982,20.28833,16.26915,16.915216,16.981228,17.406322,21.613356,20.063362,22.91091,21.425461,22.52836,21.767059,16.986458,18.41732,17.617346,15.418739,18.456139,19.778135,19.495872,21.904484,24.225199,22.8634,24.340597,23.357067,18.582523


In [163]:
display(mae)
display(mape)

17.733992

1.3989776

In [164]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}")
            target_sum = 0
            prediction_sum = 0
            
        prediction = model.predict(test_input.iloc[i:i+1])
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [165]:
errors, mae, mape = year_mae_mape_calculator(trained_model, 
                                             test_input, 
                                             test_target, 
                                             167)
display(errors)
display(mae)
display(mape)

1/1 [==============================] - 0s 13ms/step
Ano-0: |Prediction[[0.07094908]] - Target[150.491]| =  Error: [[150.42004]]
1/1 [==============================] - 0s 12ms/step
Ano-0: |Prediction[[-86.45922]] - Target[148.69600000000003]| =  Error: [[235.15521]]
1/1 [==============================] - 0s 13ms/step
Ano-11: |Prediction[[-90.65082]] - Target[153.80599999999998]| =  Error: [[244.45682]]


[array([[150.42004]], dtype=float32),
 array([[235.15521]], dtype=float32),
 array([[244.45682]], dtype=float32)]

210.0107

1.3901206